In [0]:
import os
from pathlib import Path
import requests
import sys

## 1. Authentication

In [0]:
os.environ['IAM_USER'] = "thedatageek"
os.environ['IAM_PWD'] = 'mayukhsobo'

## 2. Download Data

#### 2.1 Download Links

In [0]:
LINES_URL = 'http://www.fki.inf.unibe.ch/DBs/iamDB/data/lines/lines.tgz'
SENTENCE_URL = 'http://www.fki.inf.unibe.ch/DBs/iamDB/data/sentences/sentences.tgz'
WORDS_URL = 'http://www.fki.inf.unibe.ch/DBs/iamDB/data/words/words.tgz'
ASCII_URL = 'http://www.fki.inf.unibe.ch/DBs/iamDB/data/ascii/ascii.tgz'
SPLIT_URL = 'http://www.fki.inf.unibe.ch/DBs/iamDB/tasks/largeWriterIndependentTextLineRecognitionTask.zip'

DOWNLOAD = [
      LINES_URL,
      ASCII_URL,
      SPLIT_URL,
]

#### 2.2 Download the data

In [0]:
try:
  os.mkdir('data')
except FileExistsError:
  pass
DATA_DIR = Path('.').joinpath('data')

In [0]:
def _download(url, filename):
    with open(filename, 'wb') as f:
        response = requests.get(url, stream=True, auth=(os.environ.get('IAM_USER'), os.environ.get('IAM_PWD')))
        if not response.ok:
          return False
        total = response.headers.get('content-length')

        if total is None:
            f.write(response.content)
        else:
            downloaded = 0
            total = int(total)
            t = total
            c = 'Bytes'
            if total / 1024 > 1024:
             t = total/1024
             c = 'KB'
            if t > 1024:
              t = t / 1024
              c = 'MB'
            if t > 1024:
              t = t / 1024
              c = 'GB'

            sys.stdout.write(f'[*] Downloading {filename.name} of total size {round(t, 2)} {c}\n')
            for data in response.iter_content(chunk_size=max(int(total/1000), 1024*1024)):
                downloaded += len(data)
                f.write(data)
                done = int(50*downloaded/total)
                sys.stdout.write('\r[{}/{}][{}{}]'.format(downloaded, total, '█' * done, '.' * (50-done)))
                sys.stdout.flush()
    sys.stdout.write('\n')
    return True

def download(data_dir: Path):
  """
  Download the files in the directory from the mentioned
  download urls

  Args:
    data_dir: Path object to mention where the file to be downloaded
  """
  if data_dir.exists():
    print('Skipping Download..')
  else:
    data_dir.mkdir(exist_ok=True)
    print('Downloading IAM dataset...')
    for url in DOWNLOAD:
      if _download(url, data_dir.joinpath(os.path.basename(url))):
        pass
      else:
        print(f'Failed to load data form {url}')

In [172]:
download(DATA_DIR.joinpath('iam'))

[*] Downloading lines.tgz of total size 637.18 MB
[668131555/668131555][██████████████████████████████████████████████████]
[*] Downloading ascii.tgz of total size 2.49 MB
[2614394/2614394][██████████████████████████████████████████████████]
[*] Downloading largeWriterIndependentTextLineRecognitionTask.zip of total size 22.47 MB
[23009/23009][██████████████████████████████████████████████████]


In [178]:
!tree data

data
└── iam
    ├── ascii.tgz
    ├── largeWriterIndependentTextLineRecognitionTask.zip
    └── lines.tgz

1 directory, 3 files
